In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://downloadcenter.samsung.com/content/PM/202406/20240613175245756/KO/OID63540/OID63540_IB_WF8000CK-AD_BEST_SimpleUX_WF25CB8895_KO_KO_c6_s1.html"

# URL 요청 및 HTML 파싱
response = requests.get(url)
response.encoding = 'utf-8'  # 인코딩을 UTF-8로 설정
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

In [2]:
# 모든 table.table.table-bordered 선택 (위치에 상관없이)
tables = soup.select("table.table.table-bordered")

In [3]:
# 각 테이블에서 데이터 추출
data = []
for table in tables:
    tbody = table.find("tbody")
    if tbody is None:
        continue
    rows = tbody.find_all("tr")
    for row in rows:
        tds = row.find_all("td")
        # 최소 두 개의 td가 있어야 증상과 해결방안을 추출
        if len(tds) >= 2:
            # 증상: td > p > strong (없으면 p 또는 td 전체 텍스트로 대체)
            p_tag = tds[0].find("p")
            if p_tag:
                strong_tag = p_tag.find("strong")
                if strong_tag:
                    symptom = strong_tag.get_text(strip=True)
                else:
                    symptom = p_tag.get_text(strip=True)
            else:
                symptom = tds[0].get_text(strip=True)
            
            # 해결방안: td > ul 내의 li 항목을 추출하여 줄바꿈 문자로 결합
            ul_tag = tds[1].find("ul")
            if ul_tag:
                li_items = ul_tag.find_all("li")
                resolution = "\n".join(li.get_text(" ", strip=True) for li in li_items)
            else:
                resolution = tds[1].get_text(" ", strip=True)
            
            data.append({"증상": symptom, "해결방안": resolution})

In [4]:
print(data)

[{'증상': '전원이 들어오지 않아요.', '해결방안': '세탁기의 전원 버튼을 눌러주세요.\n전원 플러그를 뺀 후 다른 콘센트에 끼워 전원이 들어오는지 확인해 주세요.\n누전 차단기를 확인하여 ON으로 해주세요.\n멀티탭 용량을 확인해 주세요. 220 V / 16 A / 3600 W 이상 단독 사용 권장.\n220 V / 16 A / 3600 W 이상 단독 사용 권장.\n절전 콘센트인지 확인해 주세요. (절전 콘센트 상시로 변경)\n멀티탭의 허용 소비 전류 및 전력 부족으로 전원 꺼짐이 발생할 수 있습니다.'}, {'증상': '처음 사용 시 이상한 냄새가 나요.', '해결방안': '세탁물 투입구에 부착된 고무로 인한 냄새입니다. 고무 고유의 냄새이므로 세탁기를 계속 사용하면 냄새가 자연히 사라집니다.'}, {'증상': '동작이 되지 않아요.', '해결방안': '세탁기 전원이 연결되어 있는지 확인하세요.\n세탁기 문이 제대로 닫혀있는지 확인하세요.\n수도꼭지가 열려있는지 확인하세요.\n기본 화면에서 동작/일시정지 버튼을 다시 누르세요.'}, {'증상': '세탁 중 동작을 멈추고 급수를 해요.', '해결방안': '옷감의 종류에 따라 물을 흡수하는 세탁물이 있어 세탁 시 물의 양을 보충해 주기 위해 급수하는 것입니다.\n거품이 많이 발생하는 세제를 투입 시에 거품 제거를 위해 배수 후 재급수를 하는 것입니다. 거품이 과다 발생하는 세제는 사용하지 마세요.'}, {'증상': '급수가 되지 않아요.', '해결방안': '수도꼭지를 열었는지 확인해 주세요. 단수가 아닌지 확인해 주세요.\n급수 호스를 분리하여 급수밸브의 거름망을 청소해 주세요.\n급수 호스가 꼬이거나, 눌린 부분이 없는지 확인해 주세요.\n겨울철 결빙에 의해 수도꼭지나 급수 호스가 얼지 않았는지 확인하세요.'}, {'증상': '급수되는 물의 양이 적거나 물이 나오지 않습니다.', '해결방안': '수도꼭지를 완전히 열어주세요. 단수가 아닌지 확인해 주세요.\n급수 호스가 꼬이거나 눌려있지 않은 지 확인하